In [ ]:
#list of imports that will be required
import os
import sys

import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pandas.io.json import json_normalize
import json

%matplotlib inline
pd.set_option('display.max_columns', None)

In [ ]:
#The following code is used to read the datasets and process nested JSON data and process them into separate columns
#We can then parse all json fields and use the data as present in a flattened csv format as ususal
#credits : https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
#The original data as it is contains 4 JSON fields, ie., 'device', 'geoNetwork', 'totals', 'trafficSource'

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    #print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
#Load training data
train_df = load_df("../input/train.csv")
train_df.head()

In [ ]:
#change data types for columns with numerical values
train_df["totals.bounces"] = train_df["totals.bounces"].astype('float')
train_df["totals.hits"] = train_df["totals.hits"].astype('float')
train_df["totals.newVisits"] = train_df["totals.newVisits"].astype('float')
train_df["totals.pageviews"] = train_df["totals.pageviews"].astype('float')
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')

#We can see that there are several columns that contain NaN values, it is better to assign numeric values to them
train_df["totals.bounces"].fillna(0, inplace=True)
train_df["totals.hits"].fillna(0, inplace=True)
train_df["totals.newVisits"].fillna(0, inplace=True)
train_df["totals.pageviews"].fillna(1, inplace=True)
train_df["totals.transactionRevenue"].fillna(0, inplace=True)

In [ ]:
#DATA CLEANING

def clean_data(df):
    
    #On a cursory glance at the data, some columns have the value "not available in demo dataset". 
    #In cases where the columns contain only one type of value, we can safely drop these columns as they will not affect prediction
    print("Shape of data before dropping columns: ", df.shape)
    for col in df.columns:
        if len(df[col].unique()) == 1:
            df.drop(col,inplace=True,axis=1)
    print("Shape of data after dropping columns: ", df.shape)
    
    #modifying date to date_time type
    print("Now modifing date to date_time type")
    #df['date'] = df['date'].astype(str)
    #df["date"] = df["date"].apply(lambda x : datetime.date(int((x)[:4]), int((x)[4:6]), int((x)[6:])))
    df["date"] = pd.to_datetime(df.date, format="%Y%m%d")
    
    #lets drop the sessionId column, which is just a combination of fullVisitorId and visitId
    df = df.drop('sessionId', axis=1)
    
    #I add a few columns for day, month, day of the week, and year, so we can see how they effect the transaction amounts
    df["month"] = df['date'].dt.month
    df["year"] = df['date'].dt.year
    df["weekday"] = df['date'].dt.weekday
    
    return df

In [ ]:
train_df = clean_data(train_df)
train_df.head()

In [ ]:
#change data types for columns with numerical values
train_df["totals.bounces"] = train_df["totals.bounces"].astype('float')
train_df["totals.hits"] = train_df["totals.hits"].astype('float')
train_df["totals.newVisits"] = train_df["totals.newVisits"].astype('float')
train_df["totals.pageviews"] = train_df["totals.pageviews"].astype('float')
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')

#We can see that there are several columns that contain NaN values, it is better to assign numeric values to them
train_df["totals.bounces"].fillna(0, inplace=True)
train_df["totals.hits"].fillna(0, inplace=True)
train_df["totals.newVisits"].fillna(0, inplace=True)
train_df["totals.pageviews"].fillna(1, inplace=True)
train_df["totals.transactionRevenue"].fillna(0, inplace=True)

In [ ]:
#Looking at the data, there seem to be some cloumns still there that can be safely dropped
#If any column has a large number of entries with missing values, we can drop them
percent = (train_df.isnull().sum() / train_df.isnull().count() * 100 ).sort_values(ascending = False)
print (percent) # Returning values of nulls different of 0   

In [ ]:
#looking at this, we see that the column with missing percentage greater than 99.99% can be safely dropped.
#But other columns with a high missing percentage are still kept.
train_df = train_df.drop('trafficSource.campaignCode', axis=1)
train_df.head()

In [ ]:
train_df["totals.hits"] =  (train_df['totals.hits'] - min(train_df['totals.hits'])) / (max(train_df['totals.hits'])  - min(train_df['totals.hits']))

In [ ]:
#Some visualization...
#We can first look at the device columns and how they relate to the revenue
#we plot figures of devices and the number of non-zero revenue amounts they have

fig, axes = plt.subplots(3,2, figsize=(30,30))
train_df['device.browser'].value_counts().head(12).plot(kind='bar', ax=axes[0][0], rot=25, legend='Device Browser', color='red')
#train_df['device.browser'].value_counts().head(12).plot(kind='bar', ax=axes[0][1], rot=25, legend='Device Browser', color='red')
train_df.groupby(['device.browser'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[0][1], rot=25, legend='Device Browser', color='red')
train_df['device.deviceCategory'].value_counts().head(10).plot(kind='bar', ax=axes[1][0], rot=25, legend='Device Browser', color='blue')
train_df.groupby(['device.deviceCategory'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[1][1], rot=25, legend='Device Browser', color='blue')
train_df['device.operatingSystem'].value_counts().head(10).plot(kind='bar', ax=axes[2][0], rot=25, legend='Device Browser', color='green')
train_df.groupby(['device.operatingSystem'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[2][1], rot=25, legend='Device Browser', color='green')

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(30,30))
train_df['month'].value_counts().head(12).plot(kind='bar', ax=axes[0][0], rot=25, legend='Device Browser', color='red')
#train_df['device.browser'].value_counts().head(12).plot(kind='bar', ax=axes[0][1], rot=25, legend='Device Browser', color='red')
train_df.groupby(['month'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[0][1], rot=25, legend='Device Browser', color='red')
train_df['year'].value_counts().head(10).plot(kind='bar', ax=axes[1][0], rot=25, legend='Device Browser', color='blue')
train_df.groupby(['year'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[1][1], rot=25, legend='Device Browser', color='blue')
train_df['weekday'].value_counts().head(10).plot(kind='bar', ax=axes[2][0], rot=25, legend='Device Browser', color='green')
train_df.groupby(['weekday'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[2][1], rot=25, legend='Device Browser', color='green')

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(10,10))
#train_df["visitNumber"] = train_df["visitNumber"].astype('float')
train_df['visitNumber'].value_counts().head(25).plot(kind='line', rot=25, legend='Device Browser', color='red')
#train_df.groupby(['visitNumber'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(15).plot(kind='bar', ax=axes[0][1], rot=25, legend='Device Browser', color='green')

In [ ]:
fig, axes = plt.subplots(1,1, figsize=(10,10))
train_df.groupby(['visitNumber'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(30).plot(kind='bar', rot=25, legend='Device Browser', color='green')

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(20,20))
train_df['geoNetwork.continent'].value_counts().head(12).plot(kind='bar', ax=axes[0][0], rot=25, legend='Device Browser', color='red')
train_df.groupby(['geoNetwork.continent'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[0][1], rot=25, legend='Device Browser', color='red')
train_df['geoNetwork.subContinent'].value_counts().head(10).plot(kind='bar', ax=axes[1][0], rot=25, legend='Device Browser', color='blue')
train_df.groupby(['geoNetwork.subContinent'])['totals.transactionRevenue'].agg('sum').sort_values(ascending=False).head(10).plot(kind='bar', ax=axes[1][1], rot=25, legend='Device Browser', color='blue')

In [ ]:
#For the prediction of the top ten buyers, I add a new column to the training set, which simply checks, whether a user has made a purchase
train_df['buy'] = np.where(train_df['totals.transactionRevenue']>0, 1, 0)
train_df.head()

In [ ]:
#Modeling the dataset
#Before starting, we should first check whether the columns in the training and test sets are the same, with the exception of the transactionRevenue column
test_df = load_df("../input/test.csv")
test_df.head()

In [ ]:
test_df = clean_data(test_df)
test_df.head()

In [ ]:
#change data types for columns with numerical values
test_df["totals.bounces"] = test_df["totals.bounces"].astype('float')
test_df["totals.hits"] = test_df["totals.hits"].astype('float')
test_df["totals.newVisits"] = test_df["totals.newVisits"].astype('float')
test_df["totals.pageviews"] = test_df["totals.pageviews"].astype('float')

#We can see that there are several columns that contain NaN values, it is better to assign numeric values to them
test_df["totals.bounces"].fillna(0, inplace=True)
test_df["totals.hits"].fillna(0, inplace=True)
test_df["totals.newVisits"].fillna(0, inplace=True)
test_df["totals.pageviews"].fillna(0, inplace=True)

In [ ]:
print("Variables not in test but in train : ", set(train_df.columns).difference(set(test_df.columns)))

In [ ]:
from sklearn.preprocessing import LabelEncoder

#handle categorical columns
cat_cols = ['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem',
            'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
            'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'trafficSource.adContent', 
            'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.page', 
            'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.isTrueDirect', 'trafficSource.keyword', 'trafficSource.medium', 
            'trafficSource.referralPath', 'trafficSource.source']
for c in cat_cols:
    lbl_enc = LabelEncoder()
    train_vals = list(train_df[c].values.astype(str))
    test_vals = list(test_df[c].values.astype(str))
    lbl_enc.fit(train_vals + test_vals)
    train_df[c] = lbl_enc.transform(train_vals)
    test_df[c] = lbl_enc.transform(test_vals)

In [ ]:
train_df["totals.pageviews"] =  (train_df['totals.pageviews'] - min(train_df['totals.pageviews'])) / (max(train_df['totals.pageviews'])  - min(train_df['totals.pageviews']))
test_df["totals.hits"] =  (test_df['totals.hits'] - min(test_df['totals.hits'])) / (max(test_df['totals.hits'])  - min(test_df['totals.hits']))
test_df["totals.pageviews"] =  (test_df['totals.pageviews'] - min(test_df['totals.pageviews'])) / (max(test_df['totals.pageviews'])  - min(test_df['totals.pageviews']))

In [ ]:
#https://github.com/Microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py
import lightgbm as lgb
# specify your configurations as a dict
params = {
    'task': 'train',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 40,
    'learning_rate': 0.05,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.7,
    'bagging_freq': 5
}

feature_cols = ['channelGrouping', 'device.browser', 'device.deviceCategory', 'device.operatingSystem',
            'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro',
            'geoNetwork.networkDomain', 'geoNetwork.region', 'geoNetwork.subContinent', 'trafficSource.adContent', 
            'trafficSource.adwordsClickInfo.adNetworkType', 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.adwordsClickInfo.page', 
            'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'trafficSource.isTrueDirect', 'trafficSource.keyword', 'trafficSource.medium', 
            'trafficSource.referralPath', 'trafficSource.source', 'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews']

temp=train_df["totals.transactionRevenue"]
train_df["totals.transactionRevenue"] = np.log1p(train_df["totals.transactionRevenue"].astype(float))

In [ ]:
from sklearn.model_selection import train_test_split
#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
train_x, val_x, train_y, val_y = train_test_split(train_df[feature_cols], train_df['totals.transactionRevenue'], train_size=0.75)

In [ ]:
lgbmtrain = lgb.Dataset(train_x, label=train_y)
lgbmval = lgb.Dataset(val_x, label=val_y)
model = lgb.train(params, lgbmtrain, 1000, valid_sets=[lgbmval], early_stopping_rounds=50, verbose_eval=100)

In [ ]:
sub_df = pd.DataFrame({"fullVisitorId":test_df.fullVisitorId})

In [ ]:
from sklearn import metrics

pred = model.predict(test_df[feature_cols], num_iteration=model.best_iteration)
test_df["PredictedLogRevenue"] = np.expm1(pred)
sub_df = test_df.groupby("fullVisitorId").agg({"PredictedLogRevenue" : "sum"}).reset_index()
sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
sub_df.describe()

In [ ]:
#we cannot have negative values, so we make them 0
sub_df['PredictedLogRevenue'] = sub_df['PredictedLogRevenue'].apply(lambda x: 0 if x<0 else x)
sub_df.describe()

In [ ]:
sub_df.to_csv("final_result.csv", index=False)